In [9]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os
import datetime as dt


from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)
import scripts.functions as fn

from scripts.indicators import Fibonacci
from scripts.indicators import resample, zigzag, fibonacci_retroceso, fibonacci_extension

from binance.client import Client

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app
C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [10]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [27]:
environment = 'klines'
#environment = 'binance'

if environment == 'klines':

	klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
	with open(klines_file, 'rb') as file:
		df = pickle.load(file)
		#df.set_index('datetime',inplace=True)

	df = resample(df,(5))
else:
	#Conexion con Binance sin API-KEY
	client = Client()

	start = '2024-6-01'
	end = '2024-11-24'
	symbol = 'BTCUSDT'
	interval = '1d'
	real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	utc_start = real_start+' 00:00:00'
	utc_end = real_end+' 23:59:59'
	df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)


#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
df = df[2500:4500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


In [32]:

# Parámetros de la estrategia
short_window = 14  # Ventana corta
long_window = 28   # Ventana larga
k = 5              # Vecinos más cercanos (kNN)

# Cálculo de indicadores (RSI, CCI, ROC)
def calculate_rsi(series, window):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_cci(df, window):
    tp = (df['high'] + df['low'] + df['close']) / 3
    ma = tp.rolling(window=window).mean()
    mad = (tp - ma).abs().rolling(window=window).mean()
    return (tp - ma) / (0.015 * mad)

def calculate_roc(series, window):
    return series.diff(window) / series.shift(window) * 100

df['RSI'] = calculate_rsi(df['close'], short_window)
df['CCI'] = calculate_cci(df, long_window)
df['ROC'] = calculate_roc(df['close'], short_window)

# Normalización del volumen
df['volume_norm'] = (df['volume'] - df['volume'].min()) / (df['volume'].max() - df['volume'].min()) * 100

# Crear características de predicción
df['feature1'] = (df['RSI'] + df['CCI'] + df['ROC'] + df['volume_norm']) / 4
df['feature2'] = df['close'].rolling(short_window).mean()

# Clasificar la dirección (1 = BUY, -1 = SELL, 0 = CLEAR)
df['direction'] = np.sign(df['close'].diff(1))

# Implementar kNN para predicción
def knn_predict(features1, features2, directions, k):
    predictions = []
    for i in range(len(features1)):
        distances = np.sqrt((features1[i] - features1[:i])**2 + (features2[i] - features2[:i])**2)
        nearest_indices = np.argsort(distances)[:k]
        nearest_directions = directions[nearest_indices]
        predictions.append(np.sign(nearest_directions.sum()))
    return predictions



# Aplicar el modelo kNN
df['prediction'] = knn_predict(df['feature1'].values, df['feature2'].values, df['direction'].values, k)


# Cálculo de las EMAs para el Ribbon
df['ribbon_cross'] = 0
ema_periods = list(range(10, 50, 5))
ribbon_columns = []
for i, period in enumerate(ema_periods, start=1):
    column_name = f'ribbon_{i}'
    ribbon_columns.append(column_name)
    df[column_name] = df['close'].ewm(span=period, adjust=False).mean()

# Calcular el cruce de ribbon
def calculate_ribbon_cross(row, prev_close, ribbon_min, ribbon_max):
    if prev_close <= ribbon_max and row['close'] > ribbon_max:
        return 1
    elif prev_close >= ribbon_min and row['close'] < ribbon_min:
        return -1
    return 0

# Iterar sobre el DataFrame para calcular ribbon_cross
for idx in df.index[1:]:  # Excluir el primer índice porque no tiene anterior
    prev_idx = df.index[df.index.get_loc(idx) - 1]  # Obtener el índice anterior
    prev_close = df.at[prev_idx, 'close']
    ribbon_min = df.loc[prev_idx, ribbon_columns].min()
    ribbon_max = df.loc[prev_idx, ribbon_columns].max()
    df.at[idx, 'ribbon_cross'] = calculate_ribbon_cross(df.loc[idx], prev_close, ribbon_min, ribbon_max)

df['EMA'] = df['close'].ewm(span=100, adjust=False).mean()



# Generar señales de compra/venta
df['buy']  = np.where((df['prediction'] > 0) & (df['close']>df['EMA']) & (df['ribbon_cross']>0),df['close'],None)
df['sell'] = np.where((df['prediction'] < 0) & (df['close']>df['EMA']) & (df['ribbon_cross']>0),df['close'],None)

In [33]:
df_to_show = df #[0:20]

indicators = [
     #{'col': 'ZigZag','name': 'ZigZag','color': 'gray','row': 1, 'mode':'lines',},
     #{'col': 'long_fbe_0','name': 'long_fbe_0','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'long_fbe_1','name': 'long_fbe_1','color': 'white','row': 1, 'mode':'markers',},
     #{'col': 'fb_2','name': 'fb_2','color': 'red','row': 2, 'mode':'markers',},
     #{'col': 'stop_loss','name': 'stop_loss','color': 'gray','row': 1, 'mode':'markers',},
     #{'col': 'take_profit','name': 'take_profit','color': 'gray','row': 1, 'mode':'markers',},
        ]

    
indicators_out = [
     #{'col': 'prediction','name': 'prediction','color': 'yellow','row': 1, 'mode':'markers',},
     #{'col': 'RSI','name': 'RSI','color': 'green','row': 2, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       {'df':df_to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       {'df':df_to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()
